In [ ]:
import pandas as pd 
import pickle
import datetime
import re
import networkx as nx
import numpy as np
from tqdm import tqdm

# Atrial fibrillation (AF) patients on warfarin

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/first_mention_warfarin.pickle', 'rb') as f:
    warfarin = pickle.load(f)

In [ ]:
len(warfarin)

In [ ]:
df = pd.DataFrame(list(warfarin.items()), columns=['client_idcode', 'first_mention_warfarin']) # This code converts a dictionary (warfarin) into a Pandas DataFrame with two columns named 'client_idcode' and 'first_mention_warfarin'
df.shape

In [ ]:
df.head()

In [ ]:
ids = set(df['client_idcode'])
len(ids)

# AF patients on warfarin with diabetes

In [ ]:
def graph_from_onto(onto):
    G = nx.DiGraph()
    cl_edges = []
    for s, ts in onto.items():
        for t in ts:
            cl_edges.append((s.replace('S-', ''), t.replace('S-', '')))
    G.add_edges_from(cl_edges)
    return G

def load_onto(o):
    with open(f'/projects/data/GS/{o}', 'rb') as f:
        onto = pickle.load(f)
    G = graph_from_onto(onto)
    return G

def expand_codes(onto, codes):
    expanded = {}
    for name, top_codes in codes.items():
        expanded[name] = set()
        for c in top_codes:
            expanded[name].add(c)
            if c in onto:
                expanded[name].update(nx.ancestors(onto, c))
            else:
                print("NOT FOUND:", c, name)
        print(name, len(top_codes), len(expanded[name]))
    return expanded

In [ ]:
onto = load_onto('isa_rela_ch2pt_202009.pickle')

In [ ]:
db = {'type_1_diabetes' : ['46635009'],
      'type_2_diabetes' : ['44054006']
     }
db = expand_codes(onto,db)

In [ ]:
%%time 
with open('/projects/data/GS/pt2cui_pos_dates.pickle', 'rb') as f:
    pt2cui_pos_dates = pickle.load(f)

In [ ]:
def dates_ex(pt_data, codes):
    dates = None
    for x in codes:
        if x in pt_data and len(pt_data[x])>=2:
            if dates == None:
                dates = pt_data[x]
            else: 
                dates = dates.union(pt_data[x])
    return dates

rows = []
for pt in ids:
    pt_data = pt2cui_pos_dates.get(pt, {})
    row = {'client_idcode': pt, 'first_mention_warfarin': warfarin[pt]} 
    for concept, codes in db.items():
        dates = dates_ex(pt_data, codes)
        if dates == None:
            row[f"{concept}_date"] = np.nan
        else:
            row[f"{concept}_date"] = dates
    rows.append(row)

In [ ]:
df = pd.DataFrame(rows)
df.shape

In [ ]:
df.head()

In [ ]:
df.dropna(subset=['type_1_diabetes_date', 'type_2_diabetes_date'], how='all', inplace = True)

In [ ]:
df.head()

In [ ]:
if not row.empty:
    # Access the set within the 'Set_Column' and print it
    specific_set = row.iloc[0]['type_2_diabetes_date']
    print(f"Set for ID {specific_client_idcode}: {specific_set}")
else:
    print(f"No matching ID found for {specific_client_idcode}")

In [ ]:
df['client_idcode'].nunique()

In [ ]:
%%time
df = df[['client_idcode']].join((df[i].explode() for i in df.iloc[:,1:]))
df.shape

In [ ]:
df.head(11)

In [ ]:
for k in db.keys():
    df[f'{k}_date'] = pd.to_datetime(df[f'{k}_date']).dt.date

In [ ]:
df.head()

In [ ]:
for k in db.keys():
    df[f'{k}_delta'] = pd.to_datetime(df['first_mention_warfarin']) - pd.to_datetime(df[f'{k}_date'])

In [ ]:
df.head()

In [ ]:
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
for k in db.keys():
    t1 = df[f'{k}_delta'] >= min_time 
    t2 = df[f'{k}_delta'] <= max_time
    in_window = t1 & t2 
    df[f'{k}_in_window'] = in_window

In [ ]:
df.head()

In [ ]:
type_1_diabetes = df.loc[df['type_1_diabetes_in_window'] == True]
type_1_diabetes = type_1_diabetes[['client_idcode', 'type_1_diabetes_in_window']]
type_1_diabetes.shape

In [ ]:
type_1_diabetes.head()

In [ ]:
type_1_diabetes = set(type_1_diabetes['client_idcode'])
len(type_1_diabetes)

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/type_1_diabetes.pickle', 'wb') as f:
    pickle.dump(type_1_diabetes,f)

In [ ]:
type_2_diabetes = df.loc[df['type_2_diabetes_in_window'] == True]
type_2_diabetes = type_2_diabetes[['client_idcode', 'type_2_diabetes_in_window']]
type_2_diabetes.shape

In [ ]:
type_2_diabetes.head()

In [ ]:
type_2_diabetes = set(type_2_diabetes['client_idcode'])
len(type_2_diabetes)

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/type_2_diabetes.pickle', 'wb') as f:
    pickle.dump(type_2_diabetes,f)

# AF patients on antidiabetic drugs

In [ ]:
antidiabetic_drugs = pd.read_csv('/projects/ROCKET AF/data/antidiabetic_drugs.csv')

In [ ]:
antidiabetic_drugs.shape

In [ ]:
antidiabetic_drugs.head()

In [ ]:
del antidiabetic_drugs['Unnamed: 0']

In [ ]:
antidiabetic_drugs['first_warfarin_prescription'] = antidiabetic_drugs['client_idcode'].map(warfarin)

In [ ]:
antidiabetic_drugs = antidiabetic_drugs[antidiabetic_drugs['first_warfarin_prescription'].notna()]
antidiabetic_drugs.shape

In [ ]:
antidiabetic_drugs.head()

In [ ]:
antidiabetic_drugs['order_arrivaldtm'].isna().sum()

In [ ]:
antidiabetic_drugs['order_arrivaldtm'] = pd.to_datetime(antidiabetic_drugs['order_arrivaldtm'], utc=True).dt.date 

In [ ]:
antidiabetic_drugs['difference'] = pd.to_datetime(antidiabetic_drugs['first_warfarin_prescription']) - pd.to_datetime(antidiabetic_drugs['order_arrivaldtm'])

In [ ]:
antidiabetic_drugs.head()

In [ ]:
min_time = datetime.timedelta(days=1) 
max_time = datetime.timedelta(days=183)
t1 = antidiabetic_drugs['difference'] >= min_time 
t2 = antidiabetic_drugs['difference'] <= max_time
in_window = t1 & t2 
antidiabetic_drugs['in_window'] = in_window

In [ ]:
antidiabetic_drugs = antidiabetic_drugs.loc[antidiabetic_drugs['in_window'] == True]
antidiabetic_drugs.shape

In [ ]:
antidiabetic_drugs.head()

In [ ]:
antidiabetic_drugs['client_idcode'].nunique()

In [ ]:
antidiabetic_drugs['drug_name'] = antidiabetic_drugs['order_name'].str.upper()
antidiabetic_drugs['order_summaryline_upper_case'] = antidiabetic_drugs['order_summaryline'].str.upper()

In [ ]:
on_antidiabetic_drugs = set(antidiabetic_drugs['client_idcode'])
len(on_antidiabetic_drugs)

In [ ]:
with open('/projects/ROCKET AF/1A/warfarin dataextraction/data/on_antidiabetic_drugs.pickle', 'wb') as f:
    pickle.dump(on_antidiabetic_drugs,f)

--------------